In [332]:
import os, sys

PATH_DATA = os.path.dirname(os.path.dirname(os.getcwd()))
PATH_DATA

'/Users/johnbarrera/Documents/My_projects/Analysis/IA_risk_index'

In [420]:
import pandas as pd
import numpy as np
import shapefile as shp
import osmnx as ox
import contextily as ctx
import geopandas as gpd
import matplotlib as mpl
import matplotlib.pyplot as plt
import plotly.express as px
import matplotlib.colors as mcolors
import matplotlib.patches as mpatches
from matplotlib.lines import Line2D  # for legend handle
from matplotlib_scalebar.scalebar import ScaleBar
from sklearn.metrics.pairwise import haversine_distances
from shapely.geometry import Point
from pyproj import Proj, transform
import math
from matplotlib.colors import Normalize

sys.path.append(os.path.join(os.path.dirname(os.getcwd()), "../src"))
import functions_support as fsupport

import importlib

importlib.reload(fsupport)

import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation

import hashlib
from datetime import datetime, timedelta

from shapely import wkt


In [421]:
COUNTRY_NAME = 'peru'
RESULT_DIR = "results_INEI"

In [422]:
directory = PATH_DATA + "/data/" + COUNTRY_NAME + "/" + RESULT_DIR
if not os.path.exists(directory):
    os.makedirs(directory)
    
directory_object = f"{directory}/object"
if not os.path.exists(directory_object):
    os.makedirs(directory_object)

In [423]:
path = (
    PATH_DATA + "/data/" + COUNTRY_NAME + "/preprocessed_sources/administrative_region"
)
files = os.listdir(path)

administrative_areas_dict = dict()

for i in files:
    name = fsupport.capitalize_string(i[:-8])
    # adminitration = fsupport.capitalize_string(i[-9:-8])
    # # key_tag = f'{name} {fsupport.category_dict[adminitration]}'
    # key_tag = f'{name}{adminitration}'
    administrative_areas_dict[name] = gpd.read_file(path + "/" + i)

administrative_areas_dict.keys()

dict_keys(['Administration 2', 'Administration 3', 'Administration 1'])

In [424]:
administrative_areas_dict['Administration 3'].head()

,administration_1_code,administration_2_code,administration_3_code,administration_1,administration_2,administration_3,geometry
0,10,1009,100902,HUANUCO,PUERTO INCA,CODO DEL POZUZO,"POLYGON ((-75.31797 -9.29529, -75.31710 -9.297..."
1,10,1009,100904,HUANUCO,PUERTO INCA,TOURNAVISTA,"POLYGON ((-75.20831 -9.10175, -75.20805 -9.101..."
2,25,2503,250305,UCAYALI,PADRE ABAD,ALEXANDER VON HUMBOLDT,"POLYGON ((-74.95635 -8.79148, -74.95612 -8.792..."
3,25,2503,250302,UCAYALI,PADRE ABAD,IRAZOLA,"POLYGON ((-75.10843 -8.79990, -75.11753 -8.812..."
4,25,2503,250304,UCAYALI,PADRE ABAD,NESHUYA,"POLYGON ((-75.01589 -8.44637, -75.01585 -8.446..."


In [491]:
shock_dict = {
    "floods": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/shocks_shapes/1_floods/Áreas_de_exposición.shp",
    "mass movements": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/shocks_shapes/2_mass movements/Áreas_de_exposición.shp",
    "droughts": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/shocks_shapes/4_Sequias/sequias_severas.shp",
    "landslide": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/shocks_shapes/3_huaycos/view_av_activ_qdra.shp",
    "frost": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/shocks_shapes/5_6_Heladas_friaje/Heladas.shp",
    "extreme cold": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/shocks_shapes/5_6_Heladas_friaje/Friajes.shp",
    "snow": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/shocks_shapes/7_Snow/snow_peru.shp",
    "1-5": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/shocks_shapes_floods_return/shapes_1_mas/Flood_1in5/Flood_1in5.shp",
    "1-500": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/shocks_shapes_floods_return/shapes_1_mas/Flood_1in500/Flood_1in500.shp",
    "1-1000": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/shocks_shapes_floods_return/shapes_1_mas/Flood_1in1000/Flood_1in1000.shp",
}

resourse_dict = {
    "Banking agencies": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/peru_extra/Agencias_Bancarias/Agencias_Bancarias.shp",
    "Rural properties": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/peru_extra/Peru_rural_properties_shapefile/Rural_properties.shp",
    "National roads": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/peru_extra/Peru_national_road_pavements_GeoPackage/Peru_roads_pavement.gpkg",
    "Transmition lines": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/peru_extra/Peru_transmition_lines_shapefile/transmition_lines.shp",
    "Urban properties": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/peru_extra/Peru_urban_properties_shapefile/Urban_properties.shp",
    "Hydroelectric power plants": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/peru_extra/Peru_ Hydroelectric_power_plants_shapefile/Peru_power_plants_shapefile.shp",
    "Thermal power plants": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/peru_extra/Peru_thermal_power_plant_shapefile/Peru_thermal_power_plant.shp",
    "Water infrastructure": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/peru_extra/Peru_water_infrastructure_wells_dams_intake_dams_shapefile/Water_infrastructure_wells_dams_intake_dams.shp",
    "Antennae": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/peru_extra/ANtenas ICT.xlsx", 
    
    "Banking agencies total": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/peru_extra/Agencias_Bancarias/Agencias_Bancarias.shp",
    "Rural properties total": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/peru_extra/Peru_rural_properties_shapefile/Rural_properties.shp",
    "National roads total": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/peru_extra/Peru_national_road_pavements_GeoPackage/Peru_roads_pavement.gpkg",
    "Transmition lines total": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/peru_extra/Peru_transmition_lines_shapefile/transmition_lines.shp",
    "Urban properties total": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/peru_extra/Peru_urban_properties_shapefile/Urban_properties.shp",
    "Hydroelectric power plants total": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/peru_extra/Peru_ Hydroelectric_power_plants_shapefile/Peru_power_plants_shapefile.shp",
    "Thermal power plants total": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/peru_extra/Peru_thermal_power_plant_shapefile/Peru_thermal_power_plant.shp",
    "Water infrastructure total": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/peru_extra/Peru_water_infrastructure_wells_dams_intake_dams_shapefile/Water_infrastructure_wells_dams_intake_dams.shp",
    "Antennae total": f"{PATH_DATA}/data/{COUNTRY_NAME}/original_sources/peru_extra/ANtenas ICT.xlsx", 
    
}

In [492]:
from geopy.distance import geodesic


def calculate_distance(object_value):
    if object_value.geom_type == 'LineString':
        line = object_value
        distance_km = geodesic((line.coords[0][1], line.coords[0][0]),
                               (line.coords[1][1], line.coords[1][0])).kilometers
        return distance_km
    elif object_value.geom_type == 'MultiLineString':
        total_distance = 0.0
        for line in object_value.geoms:
            distance_km = geodesic((line.coords[0][1], line.coords[0][0]), 
                                   (line.coords[1][1], line.coords[1][0])).kilometers
            total_distance += line.length
        return total_distance
    else:
        return None


def build_gdf_resource(gdf_path, resource_name=''):
    
    if resource_name in ["Banking agencies", "Banking agencies total"]:
        gdf = gpd.read_file(gdf_path, crs=4326)
        gdf = gdf.rename(columns={"objectid": 'id'})
        gdf['id'].fillna(0, inplace=True)
        gdf = gdf[['id','geometry']]
        return gdf
    
    if resource_name in ["Rural properties", "Rural properties total"]:
        gdf = gpd.read_file(gdf_path, crs=3395)
        gdf = gdf.rename(columns={"predio": 'id'})
        gdf['id'].fillna(0, inplace=True)
        gdf = gdf[['id','geometry']]
        # gdf.set_crs(epsg=3395, inplace=True)
        gdf = gdf.to_crs(epsg=3395)
        gdf['geometry'] = gdf['geometry'].centroid
        # gdf.set_crs(epsg=4326)
        gdf = gdf.to_crs(epsg=4326)
        return gdf
    
    if resource_name in ["Urban properties", "Urban properties total"]:
        gdf = gpd.read_file(gdf_path, crs=3395)
        gdf = gdf.rename(columns={"id_predio": 'id'})
        gdf['id'].fillna(0, inplace=True)
        gdf = gdf[['id','geometry']]
        # gdf.set_crs(epsg=3395, inplace=True)
        gdf = gdf.to_crs(epsg=3395)
        gdf['geometry'] = gdf['geometry'].centroid
        # gdf.set_crs(epsg=4326)
        gdf = gdf.to_crs(epsg=4326)
        return gdf
    
    if resource_name in ["National roads", "National roads total"]:
        gdf = gpd.read_file(gdf_path, crs=4326)
        gdf = gdf.rename(columns={"roadID": 'id'})
        gdf['id'].fillna(0, inplace=True)
        gdf = gdf[['id','geometry']]
        return gdf
    
    if resource_name in ["Transmition lines", "Transmition lines total"]:
        gdf = gpd.read_file(gdf_path, crs=4326)
        gdf = gdf.rename(columns={"nombre_lin": 'id'})
        gdf['id'].fillna(0, inplace=True)
        gdf = gdf[['id','geometry']]
        return gdf
    
    if resource_name in ["Hydroelectric power plants", "Hydroelectric power plants total"]:
        gdf = gpd.read_file(gdf_path, crs=4326)
        gdf = gdf.rename(columns={"objectid": 'id'})
        gdf['id'].fillna(0, inplace=True)
        gdf = gdf[['id','geometry']]
        return gdf
    
    if resource_name in ["Thermal power plants", "Thermal power plants total"]:
        gdf = gpd.read_file(gdf_path, crs=4326)
        gdf = gdf.rename(columns={"objectid": 'id'})
        gdf['id'].fillna(0, inplace=True)
        gdf = gdf[['id','geometry']]
        return gdf
    
    if resource_name in ["Water infrastructure", "Water infrastructure total"]:
        gdf = gpd.read_file(gdf_path, crs=4326)
        gdf = gdf.rename(columns={"id_entidad": 'id'})
        gdf['id'].fillna(0, inplace=True)
        gdf = gdf[['id','geometry']]
        return gdf
    
    if resource_name in ["Antennae", "Antennae total"]:
        gdf = pd.read_excel(gdf_path)
        gdf['geometry'] = gdf.apply(lambda row: Point(row['X_COORD (LONGITUD) '], row['Y_COORD (LATITUD)']), axis=1)
        gdf = gpd.GeoDataFrame(gdf, geometry='geometry')
        gdf.set_crs(epsg=4326, inplace=True)
        gdf = gdf.rename(columns={"IDUBIGEO": 'id'})
        gdf['id'].fillna(0, inplace=True)
        gdf = gdf[['id','geometry']]
        return gdf
    
    return None


def measure_agg(gdf, resource_name):
    if resource_name in ["Banking agencies", "Urban properties", "Rural properties", "Hydroelectric power plants", 
                         "Thermal power plants", "Water infrastructure", "Antennae",
                         "Banking agencies total", "Urban properties total", "Rural properties total", "Hydroelectric power plants total", 
                         "Thermal power plants total", "Water infrastructure total", "Antennae total"]:
        gdf_temp = gdf.copy()
        gdf_temp = (
                gdf_temp.groupby(
                    [
                        # "administration_1_code",
                        # "administration_2_code",
                        "administration_3_code",
                    ]
                )
                .agg(
                    administration_1=("administration_1", "first"),
                    administration_2=("administration_2", "first"),
                    administration_3=("administration_3", "first"),
                    qty=("id", "count"),
                )
                .reset_index()
            )
        gdf_temp = gdf_temp.rename(columns={"qty": resource_name})
        return gdf_temp
    
    elif resource_name in ["National roads", "National roads total", 
                           "Transmition lines", "Transmition lines total"]:
        gdf_temp = gdf.copy()
        gdf_temp['geodesic_distance_meters'] = gdf_temp['geometry'].apply(calculate_distance)
        gdf_temp = (
                gdf_temp.groupby(
                    [
                        # "administration_1_code",
                        # "administration_2_code",
                        "administration_3_code",
                    ]
                )
                .agg(
                    administration_1=("administration_1", "first"),
                    administration_2=("administration_2", "first"),
                    administration_3=("administration_3", "first"),
                    qty=("geodesic_distance_meters", "sum"),
                )
                .reset_index()
            )
        gdf_temp = gdf_temp.rename(columns={"qty": resource_name})
        return gdf_temp
    return None



In [493]:

df_results = pd.DataFrame()
for SHOCK_NAME in shock_dict.keys():
# for SHOCK_NAME in list(shock_dict.keys())[:1]:
    print("Shock: " + SHOCK_NAME)
    path_shock_file = shock_dict[SHOCK_NAME]
    # Reading shape
    shock_shape = fsupport.shock_shape(path=path_shock_file, shock_name=SHOCK_NAME)
    df_resume_shock = pd.DataFrame(columns=['administration_3_code', 'administration_1',
                                            'administration_2', 'administration_3'])
    # for RESOURCE_NAME in list(resourse_dict.keys())[4:]:
    for RESOURCE_NAME in list(resourse_dict.keys()):
        print(" Resource: " + RESOURCE_NAME)
        path_resource_file = resourse_dict[RESOURCE_NAME]
        
        # Reading shape
        resource_shape = build_gdf_resource(path_resource_file, RESOURCE_NAME)
        
        # resource in event
        # if RESOURCE_NAME in ["Transmition lines total", "National roads total"]:
        if "total" in RESOURCE_NAME:
            #resource_shape2 = resource_shape.copy()
            pass
        else:
            resource_shape = gpd.sjoin(resource_shape, shock_shape, how="left", predicate="within")
            resource_shape = resource_shape[~(resource_shape["shock_name"].isna())].reset_index(drop=True)
            del resource_shape['index_right']
            del resource_shape['shock_name']
        
        
        # resource in event and region
        resource_shape = gpd.sjoin(resource_shape, administrative_areas_dict['Administration 3'], how="left", predicate="within")
        df_resume = measure_agg(resource_shape, RESOURCE_NAME)
        df_resume_shock = pd.merge(df_resume_shock, df_resume, on=['administration_3_code',
                                                                   'administration_1',
                                                                   'administration_2',
                                                                   'administration_3'], how='outer')
    df_resume_shock['shock name'] = SHOCK_NAME
    df_results = pd.concat([df_results, df_resume_shock])
    df_results.fillna(0, inplace=True)

path_results = f"{directory_object}/{COUNTRY_NAME}_object_adm3.csv"
df_results.to_csv(path_results, index=False)

Shock: floods
 Resource: Banking agencies
 Resource: Rural properties
 Resource: National roads
 Resource: Transmition lines
 Resource: Urban properties
 Resource: Hydroelectric power plants
 Resource: Thermal power plants
 Resource: Water infrastructure
 Resource: Antennae
 Resource: Banking agencies total
 Resource: Rural properties total
 Resource: National roads total
 Resource: Transmition lines total
 Resource: Urban properties total
 Resource: Hydroelectric power plants total
 Resource: Thermal power plants total
 Resource: Water infrastructure total
 Resource: Antennae total
Shock: mass movements
 Resource: Banking agencies
 Resource: Rural properties
 Resource: National roads
 Resource: Transmition lines
 Resource: Urban properties
 Resource: Hydroelectric power plants
 Resource: Thermal power plants
 Resource: Water infrastructure
 Resource: Antennae
 Resource: Banking agencies total
 Resource: Rural properties total
 Resource: National roads total
 Resource: Transmition line

In [494]:
df_results2 = df_results.iloc[:,4:].groupby('shock name').sum()

df_results2['Transmition lines'] = round(df_results2['Transmition lines'], 2)
df_results2['Transmition lines per'] = round(df_results2['Transmition lines']/df_results2['Transmition lines total']*100, 2)
df_results2['National roads'] = round(df_results2['National roads'], 2)
df_results2['National roads per'] = round(df_results2['National roads']/df_results2['National roads total']*100, 2)

df_results2['Banking agencies'] = round(df_results2['Banking agencies'], 2)
df_results2['Banking agencies per'] = round(df_results2['Banking agencies']/df_results2['Banking agencies total']*100, 2)
df_results2['Rural properties'] = round(df_results2['Rural properties'], 2)
df_results2['Rural properties per'] = round(df_results2['Rural properties']/df_results2['Rural properties total']*100, 2)
df_results2['Urban properties'] = round(df_results2['Urban properties'], 2)
df_results2['Urban properties per'] = round(df_results2['Urban properties']/df_results2['Urban properties total']*100, 2)
df_results2['Hydroelectric power plants'] = round(df_results2['Hydroelectric power plants'], 2)
df_results2['Hydroelectric power plants per'] = round(df_results2['Hydroelectric power plants']/df_results2['Hydroelectric power plants total']*100, 2)
df_results2['Thermal power plants'] = round(df_results2['Thermal power plants'], 2)
df_results2['Thermal power plants per'] = round(df_results2['Thermal power plants']/df_results2['Thermal power plants total']*100, 2)
df_results2['Water infrastructure'] = round(df_results2['Water infrastructure'], 2)
df_results2['Water infrastructure per'] = round(df_results2['Water infrastructure']/df_results2['Water infrastructure total']*100, 2)
df_results2['Antennae'] = round(df_results2['Antennae'], 2)
df_results2['Antennae per'] = round(df_results2['Antennae']/df_results2['Antennae total']*100, 2)
    

df_results2['National roads'] = df_results2.apply(lambda x: str(x['National roads'])+" Km ("+ str(x['National roads per'])+"%)", axis=1)
df_results2['Transmition lines'] = df_results2.apply(lambda x: str(x['Transmition lines'])+" Km ("+ str(x['Transmition lines per'])+"%)", axis=1)

df_results2['Banking agencies'] = df_results2.apply(lambda x: str(x['Banking agencies'])+" ("+ str(x['Banking agencies per'])+"%)", axis=1)
df_results2['Rural properties'] = df_results2.apply(lambda x: str(x['Rural properties'])+" ("+ str(x['Rural properties per'])+"%)", axis=1)
df_results2['Urban properties'] = df_results2.apply(lambda x: str(x['Urban properties'])+" ("+ str(x['Urban properties per'])+"%)", axis=1)
df_results2['Hydroelectric power plants'] = df_results2.apply(lambda x: str(x['Hydroelectric power plants'])+" ("+ str(x['Hydroelectric power plants per'])+"%)", axis=1)
df_results2['Thermal power plants'] = df_results2.apply(lambda x: str(x['Thermal power plants'])+" ("+ str(x['Thermal power plants per'])+"%)", axis=1)
df_results2['Water infrastructure'] = df_results2.apply(lambda x: str(x['Water infrastructure'])+" ("+ str(x['Water infrastructure per'])+"%)", axis=1)
df_results2['Antennae'] = df_results2.apply(lambda x: str(x['Antennae'])+" ("+ str(x['Antennae per'])+"%)", axis=1)


path_results = f"{directory_object}/{COUNTRY_NAME}_object_adm0.csv"
df_results2.to_csv(path_results)


In [495]:
                                                   

# df_results2['Transmition lines per'] = str(df_results2['Transmition lines per'])+"%"
# df_results2['National roads per'] = str(df_results2['National roads per'])+"%"

df_results2

,Banking agencies,Rural properties,National roads,Transmition lines,Urban properties,Hydroelectric power plants,Thermal power plants,Water infrastructure,Antennae,Banking agencies total,...,Antennae total,Transmition lines per,National roads per,Banking agencies per,Rural properties per,Urban properties per,Hydroelectric power plants per,Thermal power plants per,Water infrastructure per,Antennae per
shock name,,,,,,,,,,,,,,,,,,,,,
1-1000,117.0 (5.98%),82975.0 (7.69%),1968.81 Km (6.25%),0.33 Km (0.18%),24381.0 (5.87%),34.0 (32.69%),3.0 (6.98%),1570.0 (17.87%),7026.0 (7.04%),1955.0,...,99854.0,0.18,6.25,5.98,7.69,5.87,32.69,6.98,17.87,7.04
1-5,10.0 (0.51%),8033.0 (0.74%),169.05 Km (0.54%),0.0 Km (0.0%),1928.0 (0.46%),7.0 (6.73%),1.0 (2.33%),237.0 (2.7%),1442.0 (1.44%),1955.0,...,99854.0,0.00,0.54,0.51,0.74,0.46,6.73,2.33,2.70,1.44
1-500,105.0 (5.37%),68918.0 (6.39%),1583.04 Km (5.02%),0.1 Km (0.05%),18672.0 (4.5%),31.0 (29.81%),2.0 (4.65%),1308.0 (14.89%),6250.0 (6.26%),1955.0,...,99854.0,0.05,5.02,5.37,6.39,4.50,29.81,4.65,14.89,6.26
droughts,254.0 (12.99%),594238.0 (55.06%),8060.04 Km (25.58%),83.71 Km (45.46%),52921.0 (12.75%),41.0 (39.42%),4.0 (9.3%),3036.0 (34.55%),60462.0 (60.55%),1955.0,...,99854.0,45.46,25.58,12.99,55.06,12.75,39.42,9.30,34.55,60.55
extreme cold,139.0 (7.11%),48308.0 (4.48%),4143.68 Km (13.15%),16.4 Km (8.91%),25430.0 (6.12%),11.0 (10.58%),6.0 (13.95%),1244.0 (14.16%),12080.0 (12.1%),1955.0,...,99854.0,8.91,13.15,7.11,4.48,6.12,10.58,13.95,14.16,12.10
floods,131.0 (6.7%),58515.0 (5.42%),847.56 Km (2.69%),0.64 Km (0.35%),21118.0 (5.09%),0.0 (0.0%),0.0 (0.0%),402.0 (4.58%),416.0 (0.42%),1955.0,...,99854.0,0.35,2.69,6.70,5.42,5.09,0.00,0.00,4.58,0.42
frost,487.0 (24.91%),791682.0 (73.36%),12563.7 Km (39.87%),104.56 Km (56.79%),125821.0 (30.3%),87.0 (83.65%),5.0 (11.63%),4866.0 (55.38%),81484.0 (81.6%),1955.0,...,99854.0,56.79,39.87,24.91,73.36,30.30,83.65,11.63,55.38,81.60
landslide,197.0 (10.08%),146037.0 (13.53%),4401.27 Km (13.97%),39.0 Km (21.18%),56751.0 (13.67%),45.0 (43.27%),2.0 (4.65%),2107.0 (23.98%),32259.0 (32.31%),1955.0,...,99854.0,21.18,13.97,10.08,13.53,13.67,43.27,4.65,23.98,32.31
mass movements,67.0 (3.43%),13050.0 (1.21%),672.99 Km (2.14%),0.29 Km (0.16%),13611.0 (3.28%),1.0 (0.96%),0.0 (0.0%),63.0 (0.72%),705.0 (0.71%),1955.0,...,99854.0,0.16,2.14,3.43,1.21,3.28,0.96,0.00,0.72,0.71


In [433]:
df_resume

In [496]:
df_results3 = df_results[df_results['administration_1'].isin(['LIMA', 'CALLAO', 'PIURA', 'TACNA'])]
df_results3['administration_1'].replace('CALLAO', 'LIMA', inplace=True)
del df_results3['administration_3_code']
del df_results3['administration_3']
del df_results3['administration_2']
# del df_results3['shock name']
df_results3 = df_results3.groupby(['administration_1','shock name']).sum()

df_results3['Transmition lines'] = round(df_results3['Transmition lines'], 2)
df_results3['Transmition lines per'] = round(df_results3['Transmition lines']/df_results3['Transmition lines total']*100, 2)
df_results3['National roads'] = round(df_results3['National roads'], 2)
df_results3['National roads per'] = round(df_results3['National roads']/df_results3['National roads total']*100, 2)

df_results3['Banking agencies'] = round(df_results3['Banking agencies'], 2)
df_results3['Banking agencies per'] = round(df_results3['Banking agencies']/df_results3['Banking agencies total']*100, 2)
df_results3['Rural properties'] = round(df_results3['Rural properties'], 2)
df_results3['Rural properties per'] = round(df_results3['Rural properties']/df_results3['Rural properties total']*100, 2)
df_results3['Urban properties'] = round(df_results3['Urban properties'], 2)
df_results3['Urban properties per'] = round(df_results3['Urban properties']/df_results3['Urban properties total']*100, 2)
df_results3['Hydroelectric power plants'] = round(df_results3['Hydroelectric power plants'], 2)
df_results3['Hydroelectric power plants per'] = round(df_results3['Hydroelectric power plants']/df_results3['Hydroelectric power plants total']*100, 2)
df_results3['Thermal power plants'] = round(df_results3['Thermal power plants'], 2)
df_results3['Thermal power plants per'] = round(df_results3['Thermal power plants']/df_results3['Thermal power plants total']*100, 2)
df_results3['Water infrastructure'] = round(df_results3['Water infrastructure'], 2)
df_results3['Water infrastructure per'] = round(df_results3['Water infrastructure']/df_results3['Water infrastructure total']*100, 2)
df_results3['Antennae'] = round(df_results3['Antennae'], 2)
df_results3['Antennae per'] = round(df_results3['Antennae']/df_results3['Antennae total']*100, 2)


df_results3['National roads'] = df_results3.apply(lambda x: str(x['National roads'])+" Km ("+ str(x['National roads per'])+"%)", axis=1)
df_results3['Transmition lines'] = df_results3.apply(lambda x: str(x['Transmition lines'])+" Km ("+ str(x['Transmition lines per'])+"%)", axis=1)
  
df_results3['Banking agencies'] = df_results3.apply(lambda x: str(x['Banking agencies'])+" ("+ str(x['Banking agencies per'])+"%)", axis=1)
df_results3['Rural properties'] = df_results3.apply(lambda x: str(x['Rural properties'])+" ("+ str(x['Rural properties per'])+"%)", axis=1)
df_results3['Urban properties'] = df_results3.apply(lambda x: str(x['Urban properties'])+" ("+ str(x['Urban properties per'])+"%)", axis=1)
df_results3['Hydroelectric power plants'] = df_results3.apply(lambda x: str(x['Hydroelectric power plants'])+" ("+ str(x['Hydroelectric power plants per'])+"%)", axis=1)
df_results3['Thermal power plants'] = df_results3.apply(lambda x: str(x['Thermal power plants'])+" ("+ str(x['Thermal power plants per'])+"%)", axis=1)
df_results3['Water infrastructure'] = df_results3.apply(lambda x: str(x['Water infrastructure'])+" ("+ str(x['Water infrastructure per'])+"%)", axis=1)
df_results3['Antennae'] = df_results3.apply(lambda x: str(x['Antennae'])+" ("+ str(x['Antennae per'])+"%)", axis=1)


path_results = f"{directory_object}/{COUNTRY_NAME}_object_adm0P.csv"
df_results3.to_csv(path_results)

/var/folders/yh/mnw4_865319d05f6fkrpt6280000gn/T/ipykernel_16614/4036621937.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_results3['administration_1'].replace('CALLAO', 'LIMA', inplace=True)


In [498]:
df_results3

Banking agencies  Rural properties   
administration_1 shock name                                          
LIMA             1-1000              8.0 (0.83%)      2.0 (10.53%)  \
                 1-5                  0.0 (0.0%)        0.0 (0.0%)   
                 1-500               8.0 (0.83%)       1.0 (5.26%)   
                 droughts            4.0 (0.42%)        0.0 (0.0%)   
                 extreme cold         0.0 (0.0%)        0.0 (0.0%)   
                 floods             16.0 (1.67%)        0.0 (0.0%)   
                 frost              21.0 (2.19%)        0.0 (0.0%)   
                 landslide           4.0 (0.42%)        0.0 (0.0%)   
                 mass movements     46.0 (4.79%)        0.0 (0.0%)   
                 snow                 0.0 (0.0%)        0.0 (0.0%)   
PIURA            1-1000              3.0 (2.86%)  27818.0 (15.32%)   
                 1-5                  0.0 (0.0%)     994.0 (0.55%)   
                 1-500                2.0 (1.9%)  21294.0 (11.73%)   
                 droughts          26.0 (24.76%)  65170.0 (35.89%)   
                 extreme cold         0.0 (0.0%)        0.0 (0.0%)   
                 floods            30.0 (28.57%)  47076.0 (25.93%)   
                 frost             13.0 (12.38%)  43196.0 (23.79%)   
                 landslide            0.0 (0.0%)        0.0 (0.0%)   
                 mass movements       0.0 (0.0%)     248.0 (0.14%)   
                 snow                 0.0 (0.0%)        0.0 (0.0%)   
TACNA            1-1000               0.0 (0.0%)    1663.0 (5.81%)   
                 1-5                  0.0 (0.0%)     267.0 (0.93%)   
                 1-500                0.0 (0.0%)    1452.0 (5.07%)   
                 droughts            2.0 (8.33%)  15932.0 (55.66%)   
                 extreme cold         0.0 (0.0%)        0.0 (0.0%)   
                 floods             15.0 (62.5%)    1207.0 (4.22%)   
                 frost              5.0 (20.83%)  23160.0 (80.92%)   
                 landslide         20.0 (83.33%)  21683.0 (75.76%)   
                 mass movements      1.0 (4.17%)     979.0 (3.42%)   
                 snow                1.0 (4.17%)   6151.0 (21.49%)   

                                     National roads  Transmition lines   
administration_1 shock name                                              
LIMA             1-1000            76.06 Km (1.01%)    0.08 Km (0.29%)  \
                 1-5                5.38 Km (0.07%)      0.0 Km (0.0%)   
                 1-500             57.38 Km (0.77%)    0.08 Km (0.29%)   
                 droughts         163.17 Km (2.18%)  14.67 Km (52.97%)   
                 extreme cold         0.0 Km (0.0%)      0.0 Km (0.0%)   
                 floods           144.69 Km (1.93%)    0.16 Km (0.58%)   
                 frost            787.48 Km (10.5%)   17.5 Km (63.19%)   
                 landslide        237.12 Km (3.16%)  10.03 Km (36.22%)   
                 mass movements   499.89 Km (6.67%)    0.29 Km (1.05%)   
                 snow              43.41 Km (0.58%)    0.94 Km (3.39%)   
PIURA            1-1000             65.4 Km (3.67%)      0.0 Km (0.0%)   
                 1-5                 0.7 Km (0.04%)      0.0 Km (0.0%)   
                 1-500             45.74 Km (2.56%)      0.0 Km (0.0%)   
                 droughts        458.52 Km (25.71%)  16.39 Km (42.33%)   
                 extreme cold         0.0 Km (0.0%)      0.0 Km (0.0%)   
                 floods           168.23 Km (9.43%)    0.49 Km (1.27%)   
                 frost           200.76 Km (11.26%)  13.17 Km (34.02%)   
                 landslide            0.0 Km (0.0%)      0.0 Km (0.0%)   
                 mass movements     0.48 Km (0.03%)      0.0 Km (0.0%)   
                 snow                 0.0 Km (0.0%)      0.0 Km (0.0%)   
TACNA            1-1000             2.64 Km (0.27%)      0.0 Km (0.0%)   
                 1-5                 0.4 Km (0.04%)      0.0 Km (0.0%)   
               

In [497]:
df_results4 = df_results[(df_results['administration_1'].isin(['CALLAO'])) |
                         (df_results['administration_2'].isin(['LIMA']))]

df_results4['administration_1'].replace('CALLAO', 'LIMA', inplace=True)
del df_results4['administration_3_code']
del df_results4['administration_3']
del df_results4['administration_2']
# del df_results3['shock name']
df_results4 = df_results4.groupby(['administration_1','shock name']).sum()

df_results4['Transmition lines'] = round(df_results4['Transmition lines'], 2)
df_results4['Transmition lines per'] = round(df_results4['Transmition lines']/df_results4['Transmition lines total']*100, 2)
df_results4['National roads'] = round(df_results4['National roads'], 2)
df_results4['National roads per'] = round(df_results4['National roads']/df_results4['National roads total']*100, 2)

df_results4['Banking agencies'] = round(df_results4['Banking agencies'], 2)
df_results4['Banking agencies per'] = round(df_results4['Banking agencies']/df_results4['Banking agencies total']*100, 2)
df_results4['Rural properties'] = round(df_results4['Rural properties'], 2)
df_results4['Rural properties per'] = round(df_results4['Rural properties']/df_results4['Rural properties total']*100, 2)
df_results4['Urban properties'] = round(df_results4['Urban properties'], 2)
df_results4['Urban properties per'] = round(df_results4['Urban properties']/df_results4['Urban properties total']*100, 2)
df_results4['Hydroelectric power plants'] = round(df_results4['Hydroelectric power plants'], 2)
df_results4['Hydroelectric power plants per'] = round(df_results4['Hydroelectric power plants']/df_results4['Hydroelectric power plants total']*100, 2)
df_results4['Thermal power plants'] = round(df_results4['Thermal power plants'], 2)
df_results4['Thermal power plants per'] = round(df_results4['Thermal power plants']/df_results4['Thermal power plants total']*100, 2)
df_results4['Water infrastructure'] = round(df_results4['Water infrastructure'], 2)
df_results4['Water infrastructure per'] = round(df_results4['Water infrastructure']/df_results4['Water infrastructure total']*100, 2)
df_results4['Antennae'] = round(df_results4['Antennae'], 2)
df_results4['Antennae per'] = round(df_results4['Antennae']/df_results4['Antennae total']*100, 2)


df_results4['National roads'] = df_results4.apply(lambda x: str(x['National roads'])+" Km ("+ str(x['National roads per'])+"%)", axis=1)
df_results4['Transmition lines'] = df_results4.apply(lambda x: str(x['Transmition lines'])+" Km ("+ str(x['Transmition lines per'])+"%)", axis=1)
  
df_results4['Banking agencies'] = df_results4.apply(lambda x: str(x['Banking agencies'])+" ("+ str(x['Banking agencies per'])+"%)", axis=1)
df_results4['Rural properties'] = df_results4.apply(lambda x: str(x['Rural properties'])+" ("+ str(x['Rural properties per'])+"%)", axis=1)
df_results4['Urban properties'] = df_results4.apply(lambda x: str(x['Urban properties'])+" ("+ str(x['Urban properties per'])+"%)", axis=1)
df_results4['Hydroelectric power plants'] = df_results4.apply(lambda x: str(x['Hydroelectric power plants'])+" ("+ str(x['Hydroelectric power plants per'])+"%)", axis=1)
df_results4['Thermal power plants'] = df_results4.apply(lambda x: str(x['Thermal power plants'])+" ("+ str(x['Thermal power plants per'])+"%)", axis=1)
df_results4['Water infrastructure'] = df_results4.apply(lambda x: str(x['Water infrastructure'])+" ("+ str(x['Water infrastructure per'])+"%)", axis=1)
df_results4['Antennae'] = df_results4.apply(lambda x: str(x['Antennae'])+" ("+ str(x['Antennae per'])+"%)", axis=1)

path_results = f"{directory_object}/{COUNTRY_NAME}_object_adm0PL.csv"
df_results4.to_csv(path_results)
df_results4

/var/folders/yh/mnw4_865319d05f6fkrpt6280000gn/T/ipykernel_16614/1402069221.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_results4['administration_1'].replace('CALLAO', 'LIMA', inplace=True)


Banking agencies Rural properties   
administration_1 shock name                                         
LIMA             1-1000              6.0 (0.66%)       0.0 (0.0%)  \
                 1-5                  0.0 (0.0%)       0.0 (0.0%)   
                 1-500               6.0 (0.66%)       0.0 (0.0%)   
                 droughts             0.0 (0.0%)       0.0 (0.0%)   
                 extreme cold         0.0 (0.0%)       0.0 (0.0%)   
                 floods             16.0 (1.77%)       0.0 (0.0%)   
                 frost               6.0 (0.66%)       0.0 (0.0%)   
                 landslide            0.0 (0.0%)       0.0 (0.0%)   
                 mass movements     43.0 (4.76%)       0.0 (0.0%)   
                 snow                 0.0 (0.0%)       0.0 (0.0%)   

                                    National roads Transmition lines   
administration_1 shock name                                            
LIMA             1-1000            45.0 Km (0.77%)     0.0 Km (0.0%)  \
                 1-5               2.05 Km (0.03%)     0.0 Km (0.0%)   
                 1-500            32.33 Km (0.55%)     0.0 Km (0.0%)   
                 droughts            0.0 Km (0.0%)     0.0 Km (0.0%)   
                 extreme cold        0.0 Km (0.0%)     0.0 Km (0.0%)   
                 floods          124.55 Km (2.12%)     0.0 Km (0.0%)   
                 frost           300.96 Km (5.12%)  1.61 Km (18.67%)   
                 landslide         1.74 Km (0.03%)     0.0 Km (0.0%)   
                 mass movements   487.13 Km (8.3%)     0.0 Km (0.0%)   
                 snow                0.0 Km (0.0%)     0.0 Km (0.0%)   

                                 Urban properties Hydroelectric power plants   
administration_1 shock name                                                    
LIMA             1-1000            5265.0 (4.52%)                1.0 (50.0%)  \
                 1-5                182.0 (0.16%)                 0.0 (0.0%)   
                 1-500             3572.0 (3.07%)                1.0 (50.0%)   
                 droughts              0.0 (0.0%)                 0.0 (0.0%)   
                 extreme cold          0.0 (0.0%)                 0.0 (0.0%)   
                 floods            1684.0 (1.45%)                 0.0 (0.0%)   
                 frost           28652.0 (24.62%)               2.0 (100.0%)   
                 landslide             0.0 (0.0%)                 0.0 (0.0%)   
                 mass movements    6642.0 (5.71%)                 0.0 (0.0%)   
                 snow                  0.0 (0.0%)                 0.0 (0.0%)   

                                Thermal power plants Water infrastructure   
administration_1 shock name                                                 
LIMA             1-1000                  1.0 (25.0%)        15.0 (22.73%)  \
                 1-5                      0.0 (0.0%)          4.0 (6.06%)   
                 1-500                    0.0 (0.0%)         13.0 (19.7%)   
                 droughts                 0.0 (0.0%)           0.0 (0.0%)   
                 extreme cold             0.0 (0.0%)           0.0 (0.0%)   
                 floods                   0.0 (0.0%)          3.0 (4.55%)   
                 frost                    0.0 (0.0%)        37.0 (56.06%)   
                 landslide                0.0 (0.0%)          4.0 (6.06%)   
                 mass movements           0.0 (0.0%)         9.0 (13.64%)   
                 snow                     0.0 (0.0%)           0.0 (0.0%)   

                                     Antennae  Banking agencies total  ...   
administration_1 shock name                                            ...   
LIMA             1-1000           3.0 (1.65%)                   903.0  ...  \
                 1-5               0.0 (0.0%)                   903.0  ...   
                 1-500             2.0 (1.1%)                   903.0  ...   
                 droughts          0.0 (0.0%)                 

In [465]:
path_results

'/Users/johnbarrera/Documents/My_projects/Analysis/IA_risk_index/data/peru/results_INEI/object/peru_object_adm0PL.csv'